In [1]:
# Require Pandas version >= 2.2.2
import pandas as pd
assert pd.__version__ >= '2.2.2', 'Please update Pandas to version 2.2.2 or newer'
import pandas as pd, os, pyreadr
import warnings
import time
import numpy as  np
import pandas as pd
import pandas as pd
import visualization as viz
import data_loading as dl
import risk_pricing as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import importlib
importlib.reload(rp)
importlib.reload(viz)
importlib.reload(dl)
warnings.filterwarnings("ignore")
source_folder_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/"
modelname = "embeddings_km10_ipcs" # "dicfullmc10thr10defnob40noa0_8_hdp" # Don't forget to clean again when modelname changes
pfn = "pf36_name"
mo_window = 12*5
start_time = time.time()
# print the current working directory (for reference)
print("Current Working Directory:", os.getcwd())
os.chdir(source_folder_path)
print("New Working Directory:", os.getcwd())

Current Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src
New Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src


In [2]:
# # Creating additional models
# import embedding_tools as et
# from embedding_tools import Embeddings
# from embedding_tools import EmbeddingsHKRModel
# embeddings = Embeddings()
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_knocap", "knowledge capital", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_patents", "patents", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_intcap", "intangible capital", to_csv = True)

In [3]:
# Loading the data from load_dataframes:
amazon_nov01_short, cequity_mapper, ff3fw, ff5fw, ff3fm, ff5fm, topic_map_unlabeled, comparison_measures, stoxmo_orig, stoxda_orig, stoxwe_orig = \
    dl.load_dataframes(modelname, start_time, clean_again = False)


Tempo de execução depois de carregar os dados: 8.894287109375


In [6]:
import risk_pricing as rp
import importlib
importlib.reload(rp)
importlib.reload(viz)
quantiles = 8
pfname = "pfkk2me3mb" #'pfkk3me3mb', 'pf2me3mb', 'pf5me5mb', 'pfkk2me3mb', 'pfkki3me3mb'
cuts = np.linspace(0, 1, quantiles+1).tolist()
topic_map = viz.label_topic_map(topic_map_unlabeled, modelname, cuts = cuts)
fmb_5, indexed_df, eret_we, stoxwe_add = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff5fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
fmb_3, _, _, _ = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
fmb_1, _, _, _ = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, formula = "eretw ~ MktRF + HKR", kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
import visualization as viz
import importlib
importlib.reload(viz)
#import visualization as viz
base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
dir_path = os.path.join(base_path, f"{modelname}_{quantiles}tiles_{pfname}")
os.makedirs(dir_path, exist_ok=True)
figfolder = os.path.join(dir_path, "")
generate_plots = True
##
viz.explore_topic_map(topic_map, figfolder, generate_plots)
viz.explore_fmb([fmb_1, fmb_3, fmb_5], figfolder)
viz.explore_eret_we(eret_we, figfolder)
viz.tex_compare_kk_measures(comparison_measures, figfolder)
viz.plot_returns(stoxwe_add, figfolder)
viz.amazon_graph(amazon_nov01_short, figfolder)
viz.filecounter(figfolder)
viz.explore_stoxda(stoxda_orig, cequity_mapper, topic_map, figfolder)

In [41]:
max_kknt = 7
HKR_ret = (stoxwe_add
            .loc[:, ['yw', 'ntile_kk', 'me_group', 'topic_kk', 'eretw', 'me']]
            .dropna(subset=['topic_kk'])
            .groupby(['yw', 'ntile_kk', 'me_group'])
            .apply(lambda df: pd.Series({
                'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()}))
            .reset_index()
            .groupby(['yw', 'ntile_kk'])
            .agg(eret_mean=('eret', 'mean'))
            .reset_index()
            .pivot_table(index='yw', columns='ntile_kk', values='eret_mean', aggfunc='mean')
            .rename(columns=lambda x: f'kk{x}')
            .assign(HKR_SB=lambda df: df[f'kk{max_kknt}'] - df['kk0'])
            .reset_index()
            .loc[:, ['yw', 'HKR_SB']])
HKR_ret.columns.name = None
HKR_ret 
# HKR_ret = (stoxwe_add.dropna(subset=['topic_kk'])
#             .groupby(['yw', 'ntile_kk'])
            # .apply(lambda df: pd.Series({
            #     'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()}))
#             .reset_index()

,yw,HKR_SB
0,200627.0,-0.028851
1,200628.0,-0.039941
2,200629.0,-0.015079
3,200630.0,0.025082
4,200631.0,-0.019042
...,...,...
752,202049.0,0.014406
753,202050.0,-0.001068
754,202051.0,0.017294
755,202052.0,0.006375


In [ ]:
# dog = (stoxwe_add.dropna(subset=['topic_kk'])
#              .groupby(['yw', 'ntile_kk', 'me_group'])
#              .apply(lambda df: pd.Series({
#                  'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()})))
stoxwe_add.columns
# HKR_ret = (stoxwe_add.dropna(subset=['topic_kk'])
#             .groupby(['yw', 'ntile_kk', 'me_group'])
#             .apply(lambda df: pd.Series({
#                 'eret': (df['eretw'] * df['me']).sum() / df['me'].sum()}))
#             .reset_index()
#             .pivot_table(index='yw', columns='ntile_kk', values='eret', aggfunc='mean')
#             .rename(columns=lambda x: f'kk{x}')
#             .assign(HKR=lambda df: df[f'kk{max_kknt}'] - df['kk0'])
#             [['HKR']]
#             .reset_index())

Index(['yw', 'PERMNO', 'retw', 'y', 'gvkey', 'cusip8', 'cik_x', 'year_x',
       'crit_EXCHG', 'crit_PRC', 'crit_SHRCD', 'crit_ALL', 'max_topic',
       'topic_kk', 'year_y', 'CIK', 'CUSIP8', 'LPERMNO', 'sic', 'cik_y',
       'gvkey_y', 'conm', 'naics4', 'ind12', 'sic3', 'hi_tech', 'Skill',
       'xi_yeartotal', 'xi_cumsum', 'K_int_Know', 'K_int', 'at', 'prcc_f',
       'prcc_c', 'ppegt', 'csho', 'ceq', 'cusip', 'exchg', 'xir_cumsum',
       'xir_total', 'ntile_kk', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF',
       'eretw', 'fiscalyear', 'mb', 'me', 'kk_share', 'pfkk3me3mb', 'pf2me3mb',
       'pf5me5mb', 'pfkk2me3mb', 'pfkki3me3mb'],
      dtype='object')

Variables created:

- amazon_nov01_short:

- cequity_mapper:

- Fama-French factors: ff3fw, ff5fw, ff3fm, ff5fm

- Topic_map_unlabeled

- Stocks vectors: stoxmo_orig, stoxwe_orig

- comparison_measures: a vector that calculates the average industry skill and firm patent intensity for each of the topics.